In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import re

In [2]:
df=pd.read_excel('Copy of prediction_data.xlsx')

In [3]:
df

,ID,Name,Role,Transcript,Resume,Reason for decision,Job Description,num_words_in_transcript
0,rivash0038,lahar singh,software engineer,**lahar singh: software engineer candidate int...,**lahar singh**\n**software engineer candidate...,"expected_experience : 9+ years, domains: e-com...","communicated ideas clearly and effectively., h...",956
1,benjry660,benjamin ryan,data engineer,interview transcript: data engineer position\n...,here's a sample resume for benjamin ryan apply...,cultural fit,we are looking for a skilled data engineer wit...,551
2,rivash0968,amisha bedi,data scientist,"**interview transcript: amisha bedi, data scie...",**candidate profile: amisha bedi**\n\n**role:*...,"expected_experience : 6-8 years, domains: heal...","lacked key technical skills for the role., nee...",612
3,rivash0623,kairav mishra,product manager,**interview transcript: product manager positi...,**kairav mishra: product manager**\n\nas a sea...,"expected_experience : 6-8 years, domains: tech...","had impressive experience and qualifications.,...",793
4,bradgr792,bradley gross,product manager,product manager interview transcript\n\ninterv...,here's a sample resume for bradley gross apply...,cultural fit,we are looking for a skilled product manager w...,665
...,...,...,...,...,...,...,...,...
95,rivash0939,charvi verma,software engineer,**interview transcript**\n\n**interviewer:** r...,**charvi verma**\n**software engineer candidat...,"expected_experience : 3-5 years, domains: e-co...","had impressive experience and qualifications.,...",961
96,rivash0073,mythily chawla,data scientist,**interview transcript**\n\n**mythily chawla:*...,**candidate profile: mythily chawla**\n\n**int...,"expected_experience : 6-8 years, domains: heal...","had insufficient experience for the position.,...",719
97,rivash0509,ehan naidu,software engineer,**interview transcript: software engineer posi...,**candidate profile: ehan naidu**\n\n**positio...,"expected_experience : 6-8 years, domains: e-co...","lacked key technical skills for the role., str...",577
98,rivash0096,saanjh chopra,software engineer,**interview transcript: saanjh chopra - softwa...,**candidate profile: saanjh chopra**\n\n**inte...,"expected_experience : 6-8 years, domains: e-co...",demonstrated limited understanding of job requ...,827


In [4]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
def clean_text(text):
    if isinstance(text, str):  # handle potential NaN values
        text = re.sub(r'[^\w\s]', '', text).lower()  # remove punctuation and lowercase
        return text
    else:
        return ""
for col in ['Transcript', 'Resume', 'Job Description', 'Reason for decision']:
    df[col] = df[col].apply(clean_text)
    
#Text Length and Word Count
df['length_of_transcript'] = df['Transcript'].str.len()
df['resume_word_count'] = df['Resume'].str.split().str.len()

#Common Words (in transcripts)
def common_words(text, n=10):
    words = text.split()
    return Counter(words).most_common(n)

df['top_transcript_words'] = df['Transcript'].apply(lambda x: common_words(x))

#TF-IDF for Text Data (Transcript and Job Description)
tfidf = TfidfVectorizer(stop_words='english')

combined_text_data = df['Transcript'].tolist() + df['Job Description'].tolist()
tfidf.fit(combined_text_data)

transcript_tfidf = tfidf.transform(df['Transcript'])
job_desc_tfidf = tfidf.transform(df['Job Description'])

tfidf_df_transcript = pd.DataFrame(transcript_tfidf.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df_job_desc = pd.DataFrame(job_desc_tfidf.toarray(), columns=tfidf.get_feature_names_out())

print("\nTF-IDF Features for Transcripts:")
print(tfidf_df_transcript.head())

print("\nTF-IDF Features for Job Descriptions:")
print(tfidf_df_job_desc.head())


TF-IDF Features for Transcripts:
   000  030  085        10  100  1000  1115  115   12  1200  ...  year  \
0  0.0  0.0  0.0  0.000000  0.0   0.0   0.0  0.0  0.0   0.0  ...   0.0   
1  0.0  0.0  0.0  0.000000  0.0   0.0   0.0  0.0  0.0   0.0  ...   0.0   
2  0.0  0.0  0.0  0.020550  0.0   0.0   0.0  0.0  0.0   0.0  ...   0.0   
3  0.0  0.0  0.0  0.000000  0.0   0.0   0.0  0.0  0.0   0.0  ...   0.0   
4  0.0  0.0  0.0  0.022062  0.0   0.0   0.0  0.0  0.0   0.0  ...   0.0   

      years      yes  york      youd     youll     youre     youve  zev  zoom  
0  0.026291  0.00000   0.0  0.020991  0.000000  0.013753  0.039239  0.0   0.0  
1  0.015986  0.00000   0.0  0.000000  0.000000  0.033450  0.000000  0.0   0.0  
2  0.028969  0.00000   0.0  0.000000  0.023129  0.015154  0.021618  0.0   0.0  
3  0.000000  0.01756   0.0  0.000000  0.000000  0.000000  0.000000  0.0   0.0  
4  0.015551  0.00000   0.0  0.000000  0.074495  0.032538  0.023209  0.0   0.0  

[5 rows x 2964 columns]

TF-IDF Features

In [6]:
df

,ID,Name,Role,Transcript,Resume,Reason for decision,Job Description,num_words_in_transcript,length_of_transcript,resume_word_count,top_transcript_words
0,rivash0038,lahar singh,software engineer,lahar singh software engineer candidate interv...,lahar singh\nsoftware engineer candidate\n\nas...,expected_experience 9 years domains ecommerce...,communicated ideas clearly and effectively had...,956,6085,437,"[(and, 51), (the, 35), (to, 32), (i, 25), (a, ..."
1,benjry660,benjamin ryan,data engineer,interview transcript data engineer position\n\...,heres a sample resume for benjamin ryan applyi...,cultural fit,we are looking for a skilled data engineer wit...,551,3246,348,"[(you, 19), (a, 18), (data, 14), (benjamin, 13..."
2,rivash0968,amisha bedi,data scientist,interview transcript amisha bedi data scientis...,candidate profile amisha bedi\n\nrole data sci...,expected_experience 68 years domains healthca...,lacked key technical skills for the role neede...,612,3561,498,"[(i, 18), (to, 18), (you, 16), (data, 15), (in..."
3,rivash0623,kairav mishra,product manager,interview transcript product manager position\...,kairav mishra product manager\n\nas a seasoned...,expected_experience 68 years domains technolo...,had impressive experience and qualifications s...,793,4996,466,"[(and, 40), (to, 26), (a, 25), (the, 21), (i, ..."
4,bradgr792,bradley gross,product manager,product manager interview transcript\n\ninterv...,heres a sample resume for bradley gross applyi...,cultural fit,we are looking for a skilled product manager w...,665,3815,267,"[(to, 27), (a, 17), (and, 17), (with, 17), (yo..."
...,...,...,...,...,...,...,...,...,...,...,...
95,rivash0939,charvi verma,software engineer,interview transcript\n\ninterviewer rohan sing...,charvi verma\nsoftware engineer candidate\n\na...,expected_experience 35 years domains ecommerc...,had impressive experience and qualifications s...,961,6008,391,"[(and, 51), (to, 30), (a, 30), (the, 25), (wit..."
96,rivash0073,mythily chawla,data scientist,interview transcript\n\nmythily chawla hi than...,candidate profile mythily chawla\n\ninterview ...,expected_experience 68 years domains healthca...,had insufficient experience for the position n...,719,4194,578,"[(you, 26), (a, 20), (and, 20), (to, 20), (in,..."
97,rivash0509,ehan naidu,software engineer,interview transcript software engineer positio...,candidate profile ehan naidu\n\nposition softw...,expected_experience 68 years domains ecommerc...,lacked key technical skills for the role strug...,577,3568,452,"[(in, 21), (and, 20), (you, 14), (ehan, 13), (..."
98,rivash0096,saanjh chopra,software engineer,interview transcript saanjh chopra software e...,candidate profile saanjh chopra\n\ninterview f...,expected_experience 68 years domains ecommerc...,demonstrated limited understanding of job requ...,827,5004,666,"[(a, 32), (the, 26), (in, 23), (to, 23), (and,..."


In [7]:
data=df

In [9]:
data.columns

Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'Reason for decision',
       'Job Description', 'num_words_in_transcript', 'length_of_transcript',
       'resume_word_count', 'top_transcript_words'],
      dtype='object')

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()
def get_sentiment_score(transcript):
    sentiment=sia.polarity_scores(transcript)
    return sentiment['compound']
def get_length_of_resume(text):
    return len(text)
def get_words_of_resume(text):
    return len(text.split())
#Sentiment_score for transcript
data['Sentiment_score']=data['Transcript'].apply(get_sentiment_score)
#Sentiment score for resume
data['Sentiment_score_1']=data['Resume'].apply(get_sentiment_score)
#length_of_resume
data['length_of_resume']=data['Resume'].apply(get_length_of_resume)
#num_words_in_resume
data['num_words_in_resume']=data['Resume'].apply(get_words_of_resume)
data
def calculate_similarity(text1,text2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
#resume_transcript_similarity
data['resume_transcript_similarity'] = data.apply(lambda x: calculate_similarity(x['Resume'], x['Transcript']), axis=1)
#transcript_jobdesc_similarity
data['transcript_jobdesc_similarity'] = data.apply(lambda x: calculate_similarity(x['Transcript'], x['Job Description']), axis=1)
#resume_jobdesc_similarity
data['resume_jobdesc_similarity'] = data.apply(lambda x: calculate_similarity(x['Resume'], x['Job Description']), axis=1)
data

,ID,Name,Role,Transcript,Resume,Reason for decision,Job Description,num_words_in_transcript,length_of_transcript,resume_word_count,top_transcript_words,Sentiment_score,Sentiment_score_1,length_of_resume,num_words_in_resume,resume_transcript_similarity,transcript_jobdesc_similarity,resume_jobdesc_similarity
0,rivash0038,lahar singh,software engineer,lahar singh software engineer candidate interv...,lahar singh\nsoftware engineer candidate\n\nas...,expected_experience 9 years domains ecommerce...,communicated ideas clearly and effectively had...,956,6085,437,"[(and, 51), (the, 35), (to, 32), (i, 25), (a, ...",0.9995,0.9981,3035,437,0.620127,0.224662,0.242888
1,benjry660,benjamin ryan,data engineer,interview transcript data engineer position\n\...,heres a sample resume for benjamin ryan applyi...,cultural fit,we are looking for a skilled data engineer wit...,551,3246,348,"[(you, 19), (a, 18), (data, 14), (benjamin, 13...",0.9986,0.9781,2621,348,0.346506,0.286099,0.250028
2,rivash0968,amisha bedi,data scientist,interview transcript amisha bedi data scientis...,candidate profile amisha bedi\n\nrole data sci...,expected_experience 68 years domains healthca...,lacked key technical skills for the role neede...,612,3561,498,"[(i, 18), (to, 18), (you, 16), (data, 15), (in...",0.9957,0.9940,3412,498,0.452672,0.155868,0.160810
3,rivash0623,kairav mishra,product manager,interview transcript product manager position\...,kairav mishra product manager\n\nas a seasoned...,expected_experience 68 years domains technolo...,had impressive experience and qualifications s...,793,4996,466,"[(and, 40), (to, 26), (a, 25), (the, 21), (i, ...",0.9987,0.9984,3247,466,0.637855,0.248129,0.206337
4,bradgr792,bradley gross,product manager,product manager interview transcript\n\ninterv...,heres a sample resume for bradley gross applyi...,cultural fit,we are looking for a skilled product manager w...,665,3815,267,"[(to, 27), (a, 17), (and, 17), (with, 17), (yo...",0.9961,0.9844,2127,267,0.455823,0.148803,0.240797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,rivash0939,charvi verma,software engineer,interview transcript\n\ninterviewer rohan sing...,charvi verma\nsoftware engineer candidate\n\na...,expected_experience 35 years domains ecommerc...,had impressive experience and qualifications s...,961,6008,391,"[(and, 51), (to, 30), (a, 30), (the, 25), (wit...",0.9997,0.9974,2733,391,0.672172,0.265277,0.275007
96,rivash0073,mythily chawla,data scientist,interview transcript\n\nmythily chawla hi than...,candidate profile mythily chawla\n\ninterview ...,expected_experience 68 years domains healthca...,had insufficient experience for the position n...,719,4194,578,"[(you, 26), (a, 20), (and, 20), (to, 20), (in,...",0.9990,0.9712,3929,578,0.455330,0.154864,0.103405
97,rivash0509,ehan naidu,software engineer,interview transcript software engineer positio...,candidate profile ehan naidu\n\nposition softw...,expected_experience 68 years domains ecommerc...,lacked key technical skills for the role strug...,577,3568,452,"[(in, 21), (and, 20), (you, 14), (ehan, 13), (...",0.9955,0.9909,3100,452,0.467847,0.141902,0.163617
98,rivash0096,saanjh chopra,software engineer,interview transcript saanjh chopra software e...,candidate profile saanjh chopra\n\ninterview f...,expected_experience 68 years domains ecommerc...,demonstrated limited understanding of job requ...,827,5004,666,"[(a, 32), (the, 26), (in, 23), (to, 23), (and,...",0.9873,0.9967,4146,666,0.506327,0.169190,0.163048


In [12]:
# Combine text from Resume, Transcript, and Job Description to create a keyword vectorizer
combined_text = data['Transcript'].tolist() + data['Resume'].tolist() + data['Job Description'].tolist()
tfidf_vectorizer = TfidfVectorizer() 
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text)

In [13]:
import pickle
print("Fitting the TF-IDF vectorizer...")
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(combined_text_data)
print(f"Vocabulary size: {len(tfidf_vectorizer.get_feature_names_out())}")

# Step 3: Save the TF-IDF Vectorizer
output_file = 'tfidf_vectorizer.pkl'
with open(output_file, 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

Fitting the TF-IDF vectorizer...
Vocabulary size: 2964


In [14]:
# Combine text from Resume, Transcript, and Job Description to create a keyword vectorizer
combined_text = data['Transcript'].tolist() + data['Resume'].tolist() + data['Job Description'].tolist()
tfidf_vectorizer = TfidfVectorizer() 
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text)
# Function to calculate keyword match using TF-IDF vectors 
def keyword_match(text):
    vec = tfidf_vectorizer.transform([text]) 
    keywords_vec = tfidf_matrix[:len(data)] 
    return cosine_similarity(vec, keywords_vec).mean()
# Apply keyword match to Transcript, Resume, and Job Description 
data['keyword_match_transcript'] = data['Transcript'].apply(keyword_match)
data['keyword_match_resume'] = data['Resume'].apply(keyword_match) 
data['keyword_match_jobdesc'] = data['Job Description'].apply(keyword_match)

In [15]:
data.columns.values

array(['ID', 'Name', 'Role', 'Transcript', 'Resume',
       'Reason for decision', 'Job Description',
       'num_words_in_transcript', 'length_of_transcript',
       'resume_word_count', 'top_transcript_words', 'Sentiment_score',
       'Sentiment_score_1', 'length_of_resume', 'num_words_in_resume',
       'resume_transcript_similarity', 'transcript_jobdesc_similarity',
       'resume_jobdesc_similarity', 'keyword_match_transcript',
       'keyword_match_resume', 'keyword_match_jobdesc'], dtype=object)

In [16]:
data['decision'] = np.random.choice(['selected', 'rejected'], size=len(data))

In [17]:
features=['ID', 'Name', 'Role', 'Transcript', 'Resume',
       'Reason for decision', 'Job Description',
       'num_words_in_transcript', 'length_of_transcript',
       'resume_word_count', 'top_transcript_words', 'Sentiment_score',
       'Sentiment_score_1', 'length_of_resume', 'num_words_in_resume',
       'resume_transcript_similarity', 'transcript_jobdesc_similarity',
       'resume_jobdesc_similarity', 'keyword_match_transcript',
       'keyword_match_resume', 'keyword_match_jobdesc']

In [39]:
import joblib
import pandas as pd
# Load trained XGBoost model
model = joblib.load("xgboost_model.pkl")  # Update with your actual model file
# Load new dataset
new_data = data  # Update with actual dataset file
# Get expected features from the trained model
expected_features = model.feature_names_in_
# Ensure new data contains only expected features
X_new = new_data[expected_features].copy()
# Convert to numeric (if needed) and handle missing values
X_new = X_new.apply(pd.to_numeric, errors='coerce')
X_new.fillna(0, inplace=True)
# Make predictions
predictions = model.predict(X_new)
# Add predictions to the dataset
new_data["decision"] = ["Selected" if pred == 1 else "Rejected" for pred in predictions]
# Save predictions to CSV
new_data.to_excel("predicted_results.xlsx", index=False)
print("Predictions saved to predicted_results.xlsx")

Predictions saved to predicted_results.xlsx


In [37]:
print(new_data['decision'].value_counts())

decision
Selected    58
Rejected    42
Name: count, dtype: int64


In [42]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

class EmailSender:
    def __init__(self, provider='gmail'):
        self.providers = {
            'gmail': {
                'smtp_server': 'smtp.gmail.com',
                'port': 587
            },
            'outlook': {
                'smtp_server': 'smtp-mail.outlook.com',
                'port': 587
            },
            'yahoo': {
                'smtp_server': 'smtp.mail.yahoo.com',
                'port': 587
            }
        }
        self.provider = provider

    def send_email(self, sender_email, sender_password, to_email, subject, body, file_path=None):
        try:
            # Configure SMTP server details
            smtp_server = self.providers[self.provider]['smtp_server']
            port = self.providers[self.provider]['port']

            # Create message
            message = MIMEMultipart()
            message["From"] = sender_email
            message["To"] = to_email
            message["Subject"] = subject
            message.attach(MIMEText(body, "plain"))

            # Attach file if provided
            if file_path and os.path.exists(file_path):
                with open(file_path, "rb") as attachment:
                    part = MIMEBase("application", "vnd.openxmlformats-officedocument.spreadsheetml.sheet")
                    part.set_payload(attachment.read())

                encoders.encode_base64(part)
                part.add_header(
                    "Content-Disposition",
                    f"attachment; filename={os.path.basename(file_path)}",
                )
                message.attach(part)

            # Send email
            with smtplib.SMTP(smtp_server, port) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.send_message(message)
                print("Email sent successfully!")
                return True

        except Exception as e:
            print(f"Email sending failed: {e}")
            return False

# Example usage
if __name__ == "__main__":
    email_sender = EmailSender(provider='gmail')  # Can change to 'outlook' or 'yahoo'
    email_sender.send_email(
        sender_email="uppariupendra11@gmail.com",
        sender_password="eseuxrzxutsyjwse",
        to_email="brave21@gmail.com",
        subject="About Interview",
        body="Please find attached the predictions output Excel file.",
        file_path="predictions_output.xlsx"
    )

Email sent successfully!
